In [ ]:
# Install and import the required packages
from distutils.dir_util import copy_tree
copy_tree("/content/drive/MyDrive/Capstone_Project/Capstone Project/results/lib", "/content/drive/MyDrive/Capstone_Project/Capstone Project/Working")
!pip install efficientnet_pytorch 

import os
import csv
import albumentations as A
from collections import defaultdict
import sys
sys.path.append("/content/drive/MyDrive/Capstone_Project/Capstone Project/results/lib")

import torch.optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

In [ ]:
pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 21.6 MB 1.4 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


In [ ]:
from data import CLASSES
from models import get_unet
from training import load_segmentation_transforms, cycle_seg_classifier, save_state
from models import CLASSIFIER_MODEL_GENERATORS
from datasets import OrthonetClassificationDataset

In [ ]:
# True to train only 1 model architecture
STOP_AFTER_EFFICIENTNET = True

In [ ]:
# Paths
CSV_TRAIN_VAL = "/content/drive/MyDrive/Capstone_Project/Capstone Project/archive/train.csv"
MODEL_DIR = "/content/drive/MyDrive/Capstone_Project/Capstone Project/Working"
DATA_PATH = "/content/drive/MyDrive/Capstone_Project/Capstone Project/archive/orthonet data new"
UNET_PATH = "/content/drive/MyDrive/Capstone_Project/Capstone Project/Trained_Model_Phase-2/seg_unet_192_0.9000270.pt"

# Other settings
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
WEIGHT_LOSS = True
FOLD = 3
BS_TRAIN = 8
BS_VAL = 8
N_WORKERS = 2
N_EPOCHS = 230
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 5e-4

USE_CLAHE = True  # Adaptive histogram normalisation after segmentation to accentuate implant features

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
results_by_model_by_epoch = defaultdict(lambda: defaultdict(list))

clahe_transform = A.Compose([A.CLAHE(p=1)]) if USE_CLAHE else None

unet_model = get_unet(1, 1)
unet_model.load_state_dict(torch.load(UNET_PATH)['state_dict'])
unet_model = unet_model.to(DEVICE)

for model_type, model_generator in CLASSIFIER_MODEL_GENERATORS.items():

    # Data
    train_transforms, test_transforms = load_segmentation_transforms()
    ds_train = OrthonetClassificationDataset('train', CSV_TRAIN_VAL, DATA_PATH, train_transforms)
    ds_val = OrthonetClassificationDataset('val', CSV_TRAIN_VAL, DATA_PATH, test_transforms)
    dl_train = DataLoader(ds_train, BS_TRAIN, shuffle=True, num_workers=N_WORKERS, pin_memory=True)
    dl_val = DataLoader(ds_val, BS_VAL, shuffle=True, num_workers=N_WORKERS, pin_memory=True)

    print(f"TRAIN")
    ds_train.stats()
    print(f"VAL")
    ds_val.stats()

Found 918 train samples from /content/drive/MyDrive/Capstone_Project/Capstone Project/archive/train.csv

Found 251 val samples from /content/drive/MyDrive/Capstone_Project/Capstone Project/archive/train.csv

TRAIN
396 unique patients
Class                                             Number of samples
Hip_SmithAndNephew_Polarstem_NilCol               51
Knee_SmithAndNephew_GenesisII                     117
Hip_Stryker_Exeter                                192
Knee_Depuy_Synthes_Sigma                          78
Hip_DepuySynthes_Corail_Collar                    102
Hip_DepuySynthes_Corail_NilCol                    128
Hip_JRIOrtho_FurlongEvolution_Collar              22
Knee_SmithAndNephew_Legion2                       29
Hip_Stryker_AccoladeII                            34
Hip_SmithAndNephew_Anthology                      88
Hip_JRIOrtho_FurlongEvolution_NilCol              22
Knee_ZimmerBiomet_Oxford                          55


VAL
98 unique patients
Class                            

In [ ]:
pip install --upgrade albumentations

     |████████████████████████████████| 102 kB 4.0 MB/s 
     |████████████████████████████████| 47.8 MB 1.2 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
    # Model
    model = model_generator(n_in=2, n_out=len(CLASSES)).to(DEVICE)
    optimizer = torch.optim.AdamW((p for p in model.parameters() if p.requires_grad), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = OneCycleLR(optimizer, max_lr=LEARNING_RATE*10, steps_per_epoch=len(dl_train), epochs=N_EPOCHS)
    train_criterion = nn.CrossEntropyLoss(weight=ds_train.get_class_weights().to(DEVICE) if WEIGHT_LOSS else None)
    test_criterion = nn.CrossEntropyLoss(weight=ds_train.get_class_weights().to(DEVICE) if WEIGHT_LOSS else None)

    # Train
    best_loss, best_path, last_save_path = 1e10, None, None

    print(f"Training {model_type}")
    for epoch in range(1, N_EPOCHS + 1):
        train_loss, train_acc = cycle_seg_classifier('train', model, unet_model, dl_train, DEVICE, train_criterion, optimizer, scheduler, stack=True, clahe_transform=clahe_transform)
        val_loss, val_acc = cycle_seg_classifier('test', model, unet_model, dl_val, DEVICE, test_criterion, optimizer, stack=True, clahe_transform=clahe_transform)

        print(f"Epoch {epoch:03d}\t\tTRAIN loss: {train_loss:.4f}\tTRAIN acc: {train_acc:.4f}\t\tVAL loss: {val_loss:.4f}{'*' if val_loss < best_loss else ''}\tVAL acc: {val_acc:.4f}")

        state = {'epoch': epoch + 1,
                 'state_dict': model.state_dict(),
                 'optimizer': optimizer.state_dict(),
                 'scheduler': scheduler}
        save_path = os.path.join(MODEL_DIR, f"segclass_s_{model_type}_{epoch}_{val_loss:.07f}.pt")
        best_loss, last_save_path = save_state(state, save_path, val_loss, best_loss, last_save_path)

        results_by_model_by_epoch[model_type]['train_loss'].append(train_loss)
        results_by_model_by_epoch[model_type]['train_acc'].append(train_acc)
        results_by_model_by_epoch[model_type]['val_loss'].append(val_loss)
        results_by_model_by_epoch[model_type]['val_acc'].append(val_acc)

    with open(os.path.join(MODEL_DIR, f"segclass_s_{model_type}_{best_loss}.txt"), 'w') as f:
        writer = csv.writer(f)
        writer.writerow(results_by_model_by_epoch[model_type].keys())
        writer.writerows(zip(*results_by_model_by_epoch[model_type].values()))
        
    if STOP_AFTER_EFFICIENTNET:
      break

Training wideresnet50
Epoch 001		TRAIN loss: 2.4044	TRAIN acc: 0.2069		VAL loss: 2.6341*	VAL acc: 0.1289
Epoch 002		TRAIN loss: 2.2080	TRAIN acc: 0.2649		VAL loss: 2.1014*	VAL acc: 0.2721
Epoch 003		TRAIN loss: 2.0067	TRAIN acc: 0.3254		VAL loss: 2.2662	VAL acc: 0.2292
Epoch 004		TRAIN loss: 1.8313	TRAIN acc: 0.3475		VAL loss: 2.1855	VAL acc: 0.2734
Epoch 005		TRAIN loss: 1.7631	TRAIN acc: 0.3851		VAL loss: 1.7894*	VAL acc: 0.3659
Epoch 006		TRAIN loss: 1.8043	TRAIN acc: 0.3504		VAL loss: 1.8702	VAL acc: 0.3620
Epoch 007		TRAIN loss: 1.7228	TRAIN acc: 0.3953		VAL loss: 2.0102	VAL acc: 0.2930
Epoch 008		TRAIN loss: 1.5796	TRAIN acc: 0.4344		VAL loss: 1.9870	VAL acc: 0.3034
Epoch 009		TRAIN loss: 1.5203	TRAIN acc: 0.4620		VAL loss: 1.7885*	VAL acc: 0.4466
Epoch 010		TRAIN loss: 1.5073	TRAIN acc: 0.4844		VAL loss: 1.8730	VAL acc: 0.3529
Epoch 011		TRAIN loss: 1.4745	TRAIN acc: 0.4993		VAL loss: 1.5252*	VAL acc: 0.5156
Epoch 012		TRAIN loss: 1.3691	TRAIN acc: 0.5272		VAL loss: 1.8417	VAL a

SyntaxError: ignored

In [ ]:
from torchsummary import summary

In [1]:
summary(model,input_size=(1,64,64))

NameError: ignored